# Calculate the Year to Date Price Trends for 10-year Treasury Notes

## Background
Treasury notes, also known as T-notes, are financial instruments that offer a fixed interest rate every six months until they mature. These instruments are available for 2, 3, 5, 7, and 10 years and can be priced above or below their face (par) value. Upon maturity, lenders are entitled to the full par value of the note. T-notes are considered coupon debt instruments, meaning lenders receive regular interest payments based on a coupon rate throughout the note’s lifespan. 

Treasury bonds, like T-notes, are debt instruments that pay a fixed interest rate every six months. Unlike T-notes, however, Treasury bonds are long-term U.S. Treasury debt instruments with terms of 20 or 30 years. Once a bond reaches maturity, the bondholder receives its face value.  Notes and bonds can be held until maturity or sold before that time.

### Pricing
Let the term of a bond (or note) be T-years with  $\lambda$ coupon payments per year; $N = \lambda{T}$ coupon payments over the bond term. Further, let $\bar{c}$ denote the annualized coupon rate, and $\bar{r}$ denote the annual effective market rate of interest. Then, the _fair price_ for the bond (or note) $V_{B}$ is the present value of coupon payments $C$ plus the discounted face (par) value $V_{P}$ of the bond:

$$
V_{B} = \frac{V_{P}}{\left(1+i\right)^{N}}+\sum_{j=1}^{N}\frac{C}{\left(1+i\right)^{j}}
$$

The coupon payment $C=\left(\bar{c}/\lambda\right)\cdot{V_{P}}$ is set when the bond is purchased, and the interest rate $i=\bar{r}/\lambda$ varies with the market (set at auction). The contract between the U.S. government, the issuer, and the bondholder (you) includes the bond’s coupon rate, maturity date, and face (par) value.

## Objective
Using historical `TMUBMUSD10Y` quotation data from the [Wall Street Journal](https://www.wsj.com/market-data/bonds/treasuries), compute the price trend for a 10-year T-Note with a coupon rate of 3.375%. We'll load the historical dataset, which contains daily Year-to-Date (YTD) Open, High, Low, and Close yield values from `01/03/23` to `06/01/23`. You'll compute the price trend from this historical data.

### Tasks and Strategies
1. Compute the T-Note price series for the daily Open, High, Low, and Close values in the historical data set. Store the T-Note price values in an array, where each row holds a date and each column corresponds to the price for the Open, High, Low, and Close values for that date.

## Setup
In the following code blocks, we set up the computational environment by including the `Include.jl` file. The `Include.jl` file loads external packages, functions that we will use in this exercise, and custom types to model the components of our problem.

In [1]:
include("Include.jl");

## Load historical data

In [2]:
path_to_data_file = joinpath(_PATH_TO_DATA,"TMUBMUSD10Y-d-01-03-23-to-06-01-23.csv");
dataset = loaddatafile(path_to_data_file);

## Compute price series

In [6]:
number_of_days = nrow(dataset)
price_array = Array{Float64,2}(undef, number_of_days, 4);

# main loop -
for i ∈ 1:number_of_days
    
end


Row,Date,Open,High,Low,Close
,Date,Float64,Float64,Float64,Float64
1,2023-06-01,3.647,3.69,3.576,3.6
2,2023-05-31,3.692,3.694,3.615,3.647
3,2023-05-30,3.806,3.806,3.687,3.692
4,2023-05-26,3.822,3.855,3.78,3.806
5,2023-05-25,3.75,3.826,3.733,3.822
6,2023-05-24,3.701,3.751,3.667,3.75
7,2023-05-23,3.715,3.759,3.69,3.701
8,2023-05-22,3.683,3.725,3.644,3.715
9,2023-05-19,3.651,3.724,3.62,3.683


In [4]:
df = DateFormat("m/d/yyyy");

In [5]:
d = Date("6/1/23",df)

0023-06-01